In [1]:
import pandas as pd

In [2]:
%%time
# warning: this code takes a long time to run, as it reads from a large file

_X = pd.read_csv("../data/training/X_meta_concat.csv", index_col=0)\
        .query("LabelName == 'Hamburger' or LabelName == 'Sandwich'")

CPU times: user 7min 23s, sys: 39.9 s, total: 8min 3s
Wall time: 8min 33s


In [6]:
_y = _X.LabelName
y, y_label_key = pd.get_dummies(_y).values, pd.get_dummies(_y).columns.values
X = _X.pipe(lambda df: df.iloc[:, 1:128*128*3 + 1].values.reshape((df.shape[0], 128, 128, 3)))

del _X
del _y

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# free up memory
del X

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
ImageDataGenerator?

TODO: apply techniques from https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
# from keras.callbacks import EarlyStopping

clf = Sequential()
clf.add(Conv2D(32, kernel_size=(3, 3),
 activation='relu',
 kernel_initializer='he_normal',
 input_shape=(128, 128, 3)))
clf.add(MaxPooling2D((2, 2)))
clf.add(Dropout(0.25))
clf.add(Conv2D(64, (3, 3), activation='relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
clf.add(Dropout(0.25))
clf.add(Conv2D(128, (3, 3), activation='relu'))
clf.add(Dropout(0.4))
clf.add(Flatten())
clf.add(Dense(128, activation='relu'))
clf.add(Dropout(0.3))
clf.add(Dense(2, activation='softmax'))
clf.compile(loss=keras.losses.categorical_crossentropy,
 optimizer=keras.optimizers.Adam(),
 metrics=['accuracy'])

In [14]:
history = clf.fit(X_train, y_train, batch_size=128, epochs=16, 
                  validation_data=(X_test, y_test))

Train on 4481 samples, validate on 498 samples
Epoch 1/16
4481/4481 [==============================] - 202s 45ms/step - loss: 9.1147 - acc: 0.4341 - val_loss: 8.6740 - val_acc: 0.4618
Epoch 2/16
4481/4481 [==============================] - 285s 64ms/step - loss: 9.1363 - acc: 0.4332 - val_loss: 8.6740 - val_acc: 0.4618
Epoch 3/16


KeyboardInterrupt: 